In [10]:
from __future__ import print_function, division
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
%matplotlib inline

from halotools.sim_manager import CachedHaloCatalog, UserSuppliedHaloCatalog
from halotools.empirical_models import HodModelFactory

from halotools.empirical_models import TrivialPhaseSpace, ZuMandelbaum15Cens, ZuMandelbaum15Sats, \
                                        Leauthaud11Cens, Leauthaud11Sats, Zheng07Cens, Zheng07Sats, \
                                        NFWPhaseSpace, SubhaloPhaseSpace

In [14]:
from halotools.mock_observables import tpcf
from halotools.mock_observables.ia_correlations import ee_3d, ed_3d

In [2]:
from halotools.empirical_models.ia_models.ia_model_components import CentralAlignment, RandomAlignment, RadialSatelliteAlignment, SubhaloAlignment
from halotools.empirical_models.ia_models.ia_strength_models import RadialSatelliteAlignmentStrength

In [32]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Eliminate halos with 0 for halo_axisA_x(,y,z)
def mask_bad_halocat(halocat):
    bad_mask = (halocat.halo_table["halo_axisA_x"] == 0) & (halocat.halo_table["halo_axisA_y"] == 0) & (halocat.halo_table["halo_axisA_z"] == 0)
    bad_mask = bad_mask ^ np.ones(len(bad_mask), dtype=bool)
    halocat._halo_table = halocat.halo_table[ bad_mask ]

In [12]:
def create_model(cen_mu, sat_mu, selected_halocat):
    cens_occ_model = Leauthaud11Cens()
    cens_prof_model = TrivialPhaseSpace()
    cens_orientation = CentralAlignment(central_alignment_strength=cen_mu)
    sats_occ_model = Leauthaud11Sats()
    prof_args = ("satellites", np.logspace(10.5, 15.2, 15))
    sats_prof_model = SubhaloPhaseSpace(*prof_args)
    sats_orientation = RadialSatelliteAlignment(satellite_alignment_strength=sat_mu, halocat=selected_halocat)

    model_instance = HodModelFactory(centrals_occupation = cens_occ_model,
                                     centrals_profile = cens_prof_model,
                                     satellites_occupation = sats_occ_model,
                                     satellites_profile = sats_prof_model,
                                     centrals_orientation = cens_orientation,
                                     satellites_orientation = sats_orientation,
                                     model_feature_calling_sequence = (
                                     'centrals_occupation',
                                     'centrals_profile',
                                     'satellites_occupation',
                                     'satellites_profile',
                                     'centrals_orientation',
                                     'satellites_orientation')
                                    )
    
    return model_instance

In [23]:
def correlate(model, rbins, halocat):
    gal_table = model.mock.galaxy_table
    coords = np.array( [ gal_table["x"], gal_table["y"], gal_table["z"] ] ).T
    orientations = np.array( [ gal_table["galaxy_axisA_x"], gal_table["galaxy_axisA_y"], gal_table["galaxy_axisA_z"] ] ).T
    
    xi = tpcf(coords, rbins, period = halocat.Lbox)
    omega = ed_3d(coords, orientations, coords, rbins, period=halocat.Lbox)
    eta = ee_3d(coords, orientations, coords, orientations, rbins, period=halocat.Lbox)
    
    return xi, omega, eta

# Shared Parameters

In [5]:
seed=132358712
cen_mu = 1.0
sat_mu = 1.0

In [16]:
rbins = np.logspace(-1,1.2,21)
rbin_centers = (rbins[:-1]+rbins[1:])/2.0

# Cached Halocat (Bolplanck)

In [8]:
cached_halocat = CachedHaloCatalog(simname='bolplanck', halo_finder='rockstar', redshift=0, version_name='halotools_v0p4')
mask_bad_halocat(cached_halocat)

In [13]:
cached_model = create_model(cen_mu, sat_mu, cached_halocat)

cached_model.populate_mock(cached_halocat,seed=seed)

/home/nvanalfen/Documents/Python Environments/IA/lib/python3.10/site-packages/halotools/empirical_models/ia_models/ia_model_components.py:190: UserWarning: `central_alignment_strength` not detected in the table, using value in self.param_dict.
  warn(msg)
/home/nvanalfen/Documents/Python Environments/IA/lib/python3.10/site-packages/halotools/empirical_models/ia_models/ia_model_components.py:520: UserWarning: `satellite_alignment_strength` key not detected in `table`.The value set in self.param_dict of this class will be used instead.
  warn(msg)


In [24]:
cached_xi, cached_omega, cached_eta = correlate(cached_model, rbins, cached_halocat)

# "User-Supplied" Halocat (Bolplanck)

In [25]:
selected_keys = ['halo_id', 'halo_upid', 'halo_mvir', 'halo_axisA_z','halo_axisA_x','halo_axisA_y', 'halo_vx','halo_vy','halo_vz', 'halo_x', 'halo_y', 'halo_z',
                'halo_mvir_host_halo', 'halo_hostid', 'halo_mpeak', 'halo_rvir', 'halo_tidal_force']

In [29]:
columns = { key : cached_halocat.halo_table[key] for key in selected_keys }

In [26]:
particle_mass = cached_halocat.particle_mass
redshift = cached_halocat.redshift
Lbox = cached_halocat.Lbox

In [30]:
user_halocat = UserSuppliedHaloCatalog(particle_mass=particle_mass, redshift=redshift, Lbox=Lbox, **columns)
mask_bad_halocat(user_halocat)

In [31]:
user_model = create_model(cen_mu, sat_mu, user_halocat)

user_model.populate_mock(user_halocat,seed=seed)

/home/nvanalfen/Documents/Python Environments/IA/lib/python3.10/site-packages/halotools/empirical_models/factories/hod_model_factory.py:741: UserWarning: 

The param_dict key smhm_delta_0 appears in more than one component model.
This is permissible, but if you are seeing this message you should be sure you understand it.
In particular, double-check that this parameter does not have conflicting meanings across components.

If you do not wish to see this message every time you instantiate, 
simply attach a _suppress_repeated_param_warning attribute 
to any of your component models and set this variable to ``True``.

  warn(msg % key)
/home/nvanalfen/Documents/Python Environments/IA/lib/python3.10/site-packages/halotools/empirical_models/factories/hod_model_factory.py:741: UserWarning: 

The param_dict key smhm_gamma_a appears in more than one component model.
This is permissible, but if you are seeing this message you should be sure you understand it.
In particular, double-check that th

# Plot Comparisons